# git clone をした場合はすでにmini_picturesディレクトリがダウンロードされているのでこのコードは実行不要です。

### もしライブラリがないなどのエラーが出た場合は
``` !pip install 〇〇 ```
と入力をすればライブラリがanacondaにインストールされて実行できるようになる
#### pipの前に!が必要であることに注意

## 2.pictureディレクトリから使用する写真を選択し、各make_modelごとにディレクトリ分けをする

In [1]:
import os, random, shutil
import pandas as pd
from sklearn.model_selection import train_test_split

pictures_path = "./pictures" #pictures = train.zip + test.zip の中身の写真がすべて入っているディレクトリ
pictures_files = os.listdir(pictures_path)

In [2]:
pictures_files_file = [f for f in pictures_files if os.path.isfile(os.path.join(pictures_path, f))]
print(pictures_files_file[:10])  # ect ["nvan03vjkj3_01.jpg", ..., ]
pictures_id = [file.split("_")[0] for file in pictures_files_file]

['0004d4463b50_01.jpg', '0004d4463b50_02.jpg', '0004d4463b50_03.jpg', '0004d4463b50_04.jpg', '0004d4463b50_05.jpg', '0004d4463b50_06.jpg', '0004d4463b50_07.jpg', '0004d4463b50_08.jpg', '0004d4463b50_09.jpg', '0004d4463b50_10.jpg']


In [6]:
len(pictures_id)

105152

In [7]:
selected_df = pd.read_csv('selected_metadata.csv',index_col=0)

In [8]:
selected_df.head()

,id,year,make,model,trim1,trim2,make_model
0,057717e667f8,2014.0,Audi,a4,A4,2.0T quattro Premium W/ Cold Weather Pkg,Audi-a4
1,05b9facbe852,2012.0,Audi,a4,A4,2.0T Premium,Audi-a4
2,0c73bef70ead,2013.0,Audi,allroad,Allroad,2.0T quattro Premium,Audi-allroad
3,0cdf5b5d0ce1,2014.0,Audi,a4,A4,2.0T quattro Premium Plus,Audi-a4
4,147eae56c94a,2014.0,Audi,q7,Q7,3.0T quattro Premium,Audi-q7


In [9]:
# selectd_pictures(写真ファイルの文字列リスト)を作成
selectd_pictures = []
for i,_id in enumerate(pictures_id):
    if _id in list(selected_df.id):
        selectd_pictures.append(pictures_files_file[i])
# print(len(selectd_pictures))
# print(selectd_pictures)

In [10]:
(train_and_valid_pictures, test_pictures, _, _) = train_test_split( selectd_pictures, selectd_pictures, test_size=0.2, random_state=0)
(train_pictures, valid_pictures, _, _) = train_test_split( train_and_valid_pictures, train_and_valid_pictures, test_size=0.25, random_state=0)

# データが統一されることを期待するので、seed=0で固定する

In [11]:
print(len(train_pictures),len(valid_pictures),len(test_pictures))

30508 10170 10170


In [12]:
#「trainデータに含まれていない、かつ、(testまたはvalid)データに含まれている」という画像が存在しないことを確認する
train_pictures_id = [file.split("_")[0] for file in train_pictures]
valid_pictures_id = [file.split("_")[0] for file in valid_pictures]
test_pictures_id = [file.split("_")[0] for file in test_pictures]

import collections
c1 = collections.Counter(train_pictures_id)
c2 = collections.Counter(valid_pictures_id)
c3 = collections.Counter(test_pictures_id)

In [13]:
def count_id_from_dict(df,c,col_name):
    for key, val in c.items():
        df.loc[df[df['id'] == key ].index,[col_name]] = val
    return df

In [16]:
selected_df["train_count"] = 0
selected_df["valid_count"] = 0
selected_df["test_count"] = 0
selected_df.head()

,id,year,make,model,trim1,trim2,make_model,train_count,valid_count,test_count
0,057717e667f8,2014.0,Audi,a4,A4,2.0T quattro Premium W/ Cold Weather Pkg,Audi-a4,0,0,0
1,05b9facbe852,2012.0,Audi,a4,A4,2.0T Premium,Audi-a4,0,0,0
2,0c73bef70ead,2013.0,Audi,allroad,Allroad,2.0T quattro Premium,Audi-allroad,0,0,0
3,0cdf5b5d0ce1,2014.0,Audi,a4,A4,2.0T quattro Premium Plus,Audi-a4,0,0,0
4,147eae56c94a,2014.0,Audi,q7,Q7,3.0T quattro Premium,Audi-q7,0,0,0


In [18]:
selected_df = count_id_from_dict(selected_df,c1,"train_count")
selected_df = count_id_from_dict(selected_df,c2,"valid_count")
selected_df = count_id_from_dict(selected_df,c3,"test_count")
selected_df.head()

In [19]:
check_data = selected_df.groupby("make_model").sum()
check_data.head()

,year,train_count,valid_count,test_count
make_model,,,,
Audi-a3,22167.0,106,42,28
Audi-a3-sportback-e-tron,2016.0,8,2,6
Audi-a4,32214.0,139,61,56
Audi-a5,18122.0,87,28,29
Audi-a6,32210.0,161,53,42


In [15]:
# selected_df.groupby("train_count").count()
check_data = selected_df.groupby("make_model").sum().sort_values("train_count", ascending=False)
print(check_data[:5])
print(sum(check_data.train_count==0)) #0にならなければならない
print(sum(check_data.valid_count==0))  #任意の数(0であることが望ましい)
print(sum(check_data.test_count==0))  #任意の数
#以上より,「trainデータに含まれていない、かつ、(testまたはvalid)データに含まれている」という画像が存在しないことを確認した。

                    year  train_count  valid_count  test_count
make_model                                                    
Honda-civic     440884.0         2070          739         695
Honda-accord    324156.0         1576          479         521
Toyota-camry    249685.0         1166          417         401
Toyota-corolla  239678.0         1148          371         385
Toyota-prius    195286.0          965          303         284
0
1
1


In [13]:
from tqdm import tqdm #進捗が分かるだけなので、なくてもよい
# selected_df...metadata.csvのdatafarame
# make_models...最終的にターゲットのクラス名となる、dataframeのカラム名
# train_pictures_id...選択した写真のidのリスト
# train_pictures...選択した写真のファイル名
# train_dir...各make_modelをまとめるディレクトリ
# pictures_path...画像コピー元のディレクトリ
def copy_and_move_dir(selected_df, make_models, pictures_path, train_pictures_id, train_pictures, train_dir):
    for make_model in tqdm(make_models):
        selectd_pictures = []
        for i,_id in enumerate(train_pictures_id):
            if _id in list(selected_df[selected_df["make_model"]==make_model].id):
                selectd_pictures.append(train_pictures[i])
        make_model_dir = os.path.join(train_dir, make_model)
        os.mkdir(make_model_dir)    
        for fname in selectd_pictures:
            src = os.path.join(pictures_path, fname)
            dst = os.path.join(make_model_dir, make_model+"_"+fname)
            shutil.copyfile(src, dst)

In [16]:
print(selected_df["make_model"].nunique())
print(selected_df["make_model"].unique())

197
['Audi-a4' 'Audi-allroad' 'Audi-q7' 'Audi-q5' 'Audi-a5' 'Audi-a3'
 'Audi-a6' 'Audi-a8-l' 'Audi-s5' 'Audi-tt-rs' 'Audi-tts' 'Audi-a7'
 'Audi-tt' 'Audi-s4' 'Audi-sq5' 'Audi-rs-5' 'Audi-a8'
 'Audi-a3-sportback-e-tron' 'Audi-q3' 'Audi-s3' 'BMW-3-series' 'BMW-x1'
 'BMW-1-series' 'BMW-5-series' 'BMW-x5' 'BMW-z4' 'BMW-2-series' 'BMW-x5-m'
 'BMW-i3' 'BMW-x6' 'BMW-x3' 'BMW-4-series' 'BMW-7-series' 'BMW-m6'
 'BMW-6-series' 'BMW-m3' 'BMW-x4' 'Honda-civic' 'Honda-accord'
 'Honda-cr-v' 'Honda-fit' 'Honda-accord-crosstour' 'Honda-accord-plug-in'
 'Honda-hr-v' 'Honda-crosstour' 'Honda-odyssey' 'Honda-accord-hybrid'
 'Honda-pilot' 'Honda-cr-z' 'Honda-element' 'Honda-ridgeline'
 'Honda-insight' 'Jeep-wrangler-unlimited' 'Jeep-compass'
 'Jeep-grand-cherokee' 'Jeep-renegade' 'Jeep-cherokee' 'Jeep-patriot'
 'Jeep-wrangler' 'Jeep-liberty' 'Lexus-es-300h' 'Lexus-gx-460'
 'Lexus-rx-350' 'Lexus-ct-200h' 'Lexus-is-350' 'Lexus-nx-300h'
 'Lexus-hs-250h' 'Lexus-lx-570' 'Lexus-es-350' 'Lexus-rc-300'
 'Lexus-nx

## selected_picturesディレクトリの作成 (400クラス分類問題を作成するためのディレクトリ)
* ただし、データが巨大すぎるためCPUでの実行は現実的ではないので、GPUが使える環境、もしくは、気長に待つことができる方のみにおすすめします。
* 他のファイルでの実行は下で作成をするmini_picturesディレクトリを用いて行います。

In [17]:
#以下,train_pictures, test_picturesを各ディレクトリ(selected_train_pictures, selected_test_pictures)に保存

# 選択した画像すべてを格納するディレクトリ
base_dir = "selected_pictures"
os.mkdir(base_dir)

train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
valid_dir = os.path.join(base_dir,'valid')
os.mkdir(valid_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

# make_models...最終的な正解のラベル名
make_models = selected_df["make_model"].value_counts().index.tolist()

copy_and_move_dir(selected_df, make_models, pictures_path, train_pictures_id, train_pictures, train_dir)
copy_and_move_dir(selected_df, make_models, pictures_path, valid_pictures_id, valid_pictures, valid_dir)
copy_and_move_dir(selected_df, make_models, pictures_path, test_pictures_id, test_pictures, test_dir)

100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [39:36<00:00, 11.91s/it]


In [18]:
check_data_ = selected_df.groupby("make_model").sum().sort_values("train_count", ascending=False)
print(check_data_[65:85].train_count.sum())
print(check_data_[65:85].valid_count.sum())
print(check_data_[65:85].test_count.sum())
print(check_data_[65:85])
mini_make_models = list(check_data_[65:85].index)

1912
657
663
                         year  train_count  valid_count  test_count
make_model                                                         
MINI-countryman       22154.0          112           31          33
Jeep-wrangler         26170.0          111           42          55
Toyota-tacoma         28185.0          111           52          61
Volkswagen-cc         22145.0          108           37          31
Audi-a3               22167.0          106           42          28
Audi-q5               22152.0          106           30          40
Mercedes-Benz-gla     22167.0          105           39          32
Mercedes-Benz-glk     22153.0          105           40          31
Mazda-mazda5          22158.0          104           36          36
BMW-x3                24161.0          103           43          46
Nissan-370z           20140.0           98           28          34
Honda-pilot           20135.0           98           26          36
Mitsubishi-outlander  20145.0      

## ミニデータセットの作成
* 今後は、今から作成をするmini_picturesディレクトリを用いて、モデルの作成を行っていきます。

In [19]:
# 2015枚のmini_train_dataと,497枚のmini_test_data （約5%のデータで20クラス分類のモデル作成をする）
# 本番では3万枚のtrain_dataと1万枚のvalid_dataと1万枚のtest_dataで400クラス分類のモデル作成
#「trainデータに含まれていない、かつ、testデータに含まれている」という画像が存在しないことを確認する -> ok
# mini_metadata.csvを作成
mini_metadata_df = pd.DataFrame(columns=selected_df.columns)
for make_model in mini_make_models:
    mini_metadata_df = pd.concat([mini_metadata_df, selected_df[selected_df["make_model"]==make_model]])
    
mini_metadata_df.reset_index(inplace=True,drop=True)
mini_metadata_df.to_csv("mini_metadata.csv")
mini_metadata_df.tail()

# 上と同様にリストから該当するidのみを取り出し、
# mini_train_pictures,mini_valid_pictures, mini_test_pictures(写真ファイルの文字列リスト)を作成
mini_train_pictures = []
for i,id_ in enumerate(train_pictures_id):
    if id_ in set(mini_metadata_df.id):
        mini_train_pictures.append(train_pictures[i])
        
mini_valid_pictures = []
for i,id_ in enumerate(valid_pictures_id):
    if id_ in list(mini_metadata_df.id):
        mini_valid_pictures.append(valid_pictures[i])
    
mini_test_pictures = []
for i,id_ in enumerate(test_pictures_id):
    if id_ in list(mini_metadata_df.id):
        mini_test_pictures.append(test_pictures[i])

In [20]:
# mini_train_pictures, mini_valid_pictures, mini_test_pictures(写真ファイルの文字列リスト)を
# mini_train, mini_test(各ディレクトリ)に保存

# 選択した画像すべてを格納するディレクトリ
mini_base_dir = "mini_pictures"
os.mkdir(mini_base_dir)

train_dir = os.path.join(mini_base_dir,'train')
os.mkdir(train_dir)
valid_dir = os.path.join(mini_base_dir,'valid')
os.mkdir(valid_dir)
test_dir = os.path.join(mini_base_dir,'test')
os.mkdir(test_dir)

# make_models...最終的にターゲットのクラス名となる、dataframeのカラム名
mini_make_models = mini_metadata_df["make_model"].value_counts().index.tolist()

copy_and_move_dir(mini_metadata_df, mini_make_models, pictures_path, train_pictures_id, train_pictures, train_dir)
copy_and_move_dir(mini_metadata_df, mini_make_models, pictures_path, valid_pictures_id, valid_pictures, valid_dir)
copy_and_move_dir(mini_metadata_df, mini_make_models, pictures_path, test_pictures_id, test_pictures, test_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:35<00:00,  7.69s/it]
